# 第1课：深度学习入门

## 学习目标
- 了解神经网络的基本概念
- 理解前向传播和反向传播
- 掌握 PyTorch 基础
- 实现简单的神经网络

## 1. 神经网络基础

### 1.1 什么是神经网络？

神经网络是一种模仿人脑结构的计算模型，由大量相互连接的节点（神经元）组成。

### 1.2 基本组成
- **输入层**：接收原始数据
- **隐藏层**：提取特征
- **输出层**：产生预测结果
- **权重和偏置**：可学习的参数
- **激活函数**：引入非线性

In [ ]:
# 安装 PyTorch: pip install torch torchvision
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

print(f"PyTorch 版本: {torch.__version__}")
print(f"CUDA 可用: {torch.cuda.is_available()}")

## 2. PyTorch 张量基础

In [ ]:
# 创建张量
# 从列表创建
x = torch.tensor([1, 2, 3, 4])
print(f"张量: {x}")
print(f"形状: {x.shape}")
print(f"数据类型: {x.dtype}")

# 从 NumPy 创建
arr = np.array([1.0, 2.0, 3.0])
y = torch.from_numpy(arr)
print(f"\n从 NumPy: {y}")

In [ ]:
# 常用创建函数
print(f"zeros: {torch.zeros(2, 3)}")
print(f"\nones: {torch.ones(2, 3)}")
print(f"\nrand: {torch.rand(2, 3)}")
print(f"\nrandn: {torch.randn(2, 3)}")

In [ ]:
# 张量运算
a = torch.tensor([[1, 2], [3, 4]], dtype=torch.float32)
b = torch.tensor([[5, 6], [7, 8]], dtype=torch.float32)

print(f"a + b:\n{a + b}")
print(f"\na * b (元素级):\n{a * b}")
print(f"\na @ b (矩阵乘法):\n{a @ b}")

In [ ]:
# 自动求导
x = torch.tensor([2.0, 3.0], requires_grad=True)
y = x ** 2 + 2 * x + 1
z = y.sum()

# 反向传播
z.backward()

print(f"x: {x}")
print(f"y = x² + 2x + 1: {y}")
print(f"dy/dx = 2x + 2: {x.grad}")

## 3. 激活函数

In [ ]:
x = torch.linspace(-5, 5, 100)

fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Sigmoid
axes[0, 0].plot(x.numpy(), torch.sigmoid(x).numpy())
axes[0, 0].set_title('Sigmoid')
axes[0, 0].grid(True)

# Tanh
axes[0, 1].plot(x.numpy(), torch.tanh(x).numpy())
axes[0, 1].set_title('Tanh')
axes[0, 1].grid(True)

# ReLU
axes[1, 0].plot(x.numpy(), torch.relu(x).numpy())
axes[1, 0].set_title('ReLU')
axes[1, 0].grid(True)

# Leaky ReLU
leaky_relu = nn.LeakyReLU(0.1)
axes[1, 1].plot(x.numpy(), leaky_relu(x).numpy())
axes[1, 1].set_title('Leaky ReLU')
axes[1, 1].grid(True)

plt.tight_layout()
plt.show()

## 4. 构建神经网络

In [ ]:
# 定义神经网络
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# 创建模型
model = SimpleNN(input_size=10, hidden_size=20, output_size=2)
print(model)

In [ ]:
# 使用 Sequential
model_seq = nn.Sequential(
    nn.Linear(10, 20),
    nn.ReLU(),
    nn.Linear(20, 10),
    nn.ReLU(),
    nn.Linear(10, 2)
)
print(model_seq)

## 5. 训练神经网络

In [ ]:
# 生成示例数据
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

X, y = make_classification(n_samples=1000, n_features=10, n_classes=2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 转换为张量
X_train_t = torch.FloatTensor(X_train)
y_train_t = torch.LongTensor(y_train)
X_test_t = torch.FloatTensor(X_test)
y_test_t = torch.LongTensor(y_test)

print(f"训练集形状: {X_train_t.shape}")

In [ ]:
# 创建模型、损失函数和优化器
model = SimpleNN(input_size=10, hidden_size=32, output_size=2)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# 训练
epochs = 100
losses = []

for epoch in range(epochs):
    # 前向传播
    outputs = model(X_train_t)
    loss = criterion(outputs, y_train_t)
    
    # 反向传播
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    losses.append(loss.item())
    
    if (epoch + 1) % 20 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

In [ ]:
# 可视化损失
plt.figure(figsize=(10, 5))
plt.plot(losses)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.grid(True)
plt.show()

In [ ]:
# 评估模型
model.eval()
with torch.no_grad():
    outputs = model(X_test_t)
    _, predicted = torch.max(outputs, 1)
    accuracy = (predicted == y_test_t).sum().item() / len(y_test_t)

print(f"测试集准确率: {accuracy:.4f}")

## 6. DataLoader 使用

In [ ]:
from torch.utils.data import DataLoader, TensorDataset

# 创建数据集
train_dataset = TensorDataset(X_train_t, y_train_t)
test_dataset = TensorDataset(X_test_t, y_test_t)

# 创建 DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print(f"批次数: {len(train_loader)}")

In [ ]:
# 使用 DataLoader 训练
model = SimpleNN(input_size=10, hidden_size=32, output_size=2)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

epochs = 50
for epoch in range(epochs):
    model.train()
    epoch_loss = 0
    
    for batch_X, batch_y in train_loader:
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
    
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {epoch_loss/len(train_loader):.4f}')

## 7. 保存和加载模型

In [ ]:
# 保存模型
torch.save(model.state_dict(), 'model.pth')
print("模型已保存")

# 加载模型
loaded_model = SimpleNN(input_size=10, hidden_size=32, output_size=2)
loaded_model.load_state_dict(torch.load('model.pth', map_location='cpu'))
loaded_model.eval()
print("模型已加载")

# 清理
import os
os.remove('model.pth')

## 8. 练习题

### 练习：构建更深的网络
创建一个包含 3 个隐藏层的神经网络

In [ ]:
# 在这里编写代码


## 9. 本课小结

1. **神经网络**：由层、权重、激活函数组成
2. **PyTorch 张量**：类似 NumPy，支持 GPU 和自动求导
3. **nn.Module**：定义神经网络的基类
4. **训练流程**：前向传播→计算损失→反向传播→更新参数
5. **DataLoader**：批量加载数据